<a href="https://colab.research.google.com/github/alina518/Projects/blob/main/Hangman_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install english_words

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 55.4 MB/s eta 0:00:00


In [4]:
!pip install pillow

In [15]:
import random
import gradio as gr
from english_words import get_english_words_set

web2flowerset = get_english_words_set(["web2"], lower=True)

# ---------- SVG ----------
def svg_frame(stage: int) -> str:
    show_head = stage >= 1
    show_body = stage >= 2
    show_left_arm = stage >= 3
    show_right_arm = stage >= 4
    show_left_leg = stage >= 4
    show_right_leg = stage >= 4

    return f"""
<div class="hang-card">
<svg viewBox="0 0 240 240" width="100%" height="300" xmlns="http://www.w3.org/2000/svg">
  <rect x="0" y="0" width="240" height="240" fill="white"/>

  <line x1="30" y1="220" x2="150" y2="220" stroke="#111827" stroke-width="6" />
  <line x1="60" y1="220" x2="60" y2="30" stroke="#111827" stroke-width="6" />
  <line x1="57" y1="30" x2="160" y2="30" stroke="#111827" stroke-width="6" />
  <line x1="160" y1="30" x2="160" y2="55" stroke="#111827" stroke-width="6" />
  <line x1="160" y1="55" x2="160" y2="70" stroke="#111827" stroke-width="4" />

  {"<circle cx='160' cy='88' r='18' fill='none' stroke='#111827' stroke-width='4' />" if show_head else ""}
  {"<line x1='160' y1='106' x2='160' y2='150' stroke='#111827' stroke-width='4' />" if show_body else ""}

  {"<line x1='160' y1='120' x2='140' y2='135' stroke='#111827' stroke-width='4' />" if show_left_arm else ""}
  {"<line x1='160' y1='120' x2='180' y2='135' stroke='#111827' stroke-width='4' />" if show_right_arm else ""}

  {"<line x1='160' y1='150' x2='145' y2='175' stroke='#111827' stroke-width='4' />" if show_left_leg else ""}
  {"<line x1='160' y1='150' x2='175' y2='175' stroke='#111827' stroke-width='4' />" if show_right_leg else ""}

  <text x="10" y="20" font-size="12" fill="#6b7280">Stage: {stage}/4</text>
</svg>
</div>
"""

# ---------- Helpers ----------
def stage_from_lives(lives: int) -> int:
    return min(4 - lives, 4)

def build_display(word: str, guessed: set) -> str:
    return " ".join([ch if ch in guessed else "_" for ch in word])

def pick_word(difficulty: str) -> str:
    if difficulty == "Easy (4–6)":
        pool = [w for w in web2flowerset if w.isalpha() and 4 <= len(w) <= 6]
    elif difficulty == "Medium (7–9)":
        pool = [w for w in web2flowerset if w.isalpha() and 7 <= len(w) <= 9]
    else:
        pool = [w for w in web2flowerset if w.isalpha() and len(w) >= 10]
    if not pool:
        pool = [w for w in web2flowerset if w.isalpha()]
    return random.choice(pool).lower()

def safe_sets(guessed, wrong):
    return set(guessed or []), set(wrong or [])

def lives_badge(lives: int) -> str:
    full = "❤️" * max(0, lives)
    empty = "🖤" * max(0, 4 - lives)
    return f"""
<div class="lives-badge" title="Lives">
  <span class="lives-label">Lives</span>
  <span class="lives-hearts">{full}{empty}</span>
</div>
"""

def status_banner(text: str, kind: str = "info") -> str:
    klass = {"info":"status-info", "good":"status-good", "bad":"status-bad"}.get(kind, "status-info")
    return f"""<div class="status {klass}">{text}</div>"""

def wrong_line(wrong_set) -> str:
    wrong = " ".join(sorted(wrong_set)) if wrong_set else "-"
    return f"""
<div class="wrong-wrap">
  <div class="wrong-title">Wrong guesses</div>
  <div class="wrong-value">{wrong}</div>
  <div class="wrong-space"></div>
</div>
"""

# ---------- Game ----------
def init_ui():
    return (
        None, [], [], 4, False, False,
        "<div class='word-display'>_ _ _ _</div>",
        svg_frame(0),
        lives_badge(4),
        status_banner("Click New Game to start.", "info"),
        wrong_line(set()),
        ""
    )

def new_game(difficulty: str):
    word = pick_word(difficulty)
    guessed = set()
    wrong = set()
    lives = 4
    over = False
    hint_used = False

    stage = stage_from_lives(lives)
    word_html = f"<div class='word-display'>{build_display(word, guessed)}</div>"

    return (
        word, list(guessed), list(wrong), lives, over, hint_used,
        word_html,
        svg_frame(stage),
        lives_badge(lives),
        status_banner("Game started. Enter a letter.", "info"),
        wrong_line(wrong),
        ""
    )

def guess_letter(user, word, guessed, wrong, lives, over, hint_used):
    guessed, wrong = safe_sets(guessed, wrong)

    if not word or not isinstance(word, str):
        return (
            word, list(guessed), list(wrong), lives, over, hint_used,
            "<div class='word-display'>_ _ _ _</div>",
            svg_frame(0),
            lives_badge(4),
            status_banner("Click New Game first.", "info"),
            wrong_line(wrong),
            ""
        )

    if over:
        stage = stage_from_lives(lives)
        return (
            word, list(guessed), list(wrong), lives, True, hint_used,
            f"<div class='word-display'>{build_display(word, guessed)}</div>",
            svg_frame(stage),
            lives_badge(lives),
            status_banner("Game over. Click New Game.", "info"),
            wrong_line(wrong),
            ""
        )

    user = (user or "").strip().lower()
    if len(user) != 1 or not user.isalpha():
        stage = stage_from_lives(lives)
        return (
            word, list(guessed), list(wrong), lives, False, hint_used,
            f"<div class='word-display'>{build_display(word, guessed)}</div>",
            svg_frame(stage),
            lives_badge(lives),
            status_banner("Enter exactly one alphabet letter.", "bad"),
            wrong_line(wrong),
            ""
        )

    if user in guessed or user in wrong:
        stage = stage_from_lives(lives)
        return (
            word, list(guessed), list(wrong), lives, False, hint_used,
            f"<div class='word-display'>{build_display(word, guessed)}</div>",
            svg_frame(stage),
            lives_badge(lives),
            status_banner("Already tried that letter.", "info"),
            wrong_line(wrong),
            ""
        )

    if user in word:
        guessed.add(user)
        msg = status_banner("Correct.", "good")
    else:
        wrong.add(user)
        lives -= 1
        msg = status_banner("Wrong.", "bad")

    stage = stage_from_lives(lives)
    word_html = f"<div class='word-display'>{build_display(word, guessed)}</div>"

    if all(ch in guessed for ch in word):
        over = True
        msg = status_banner("You won. Click New Game to play again.", "good")
    elif lives == 0 and not all(ch in guessed for ch in word):
        over = True
        msg = status_banner(f"Out of lives. The word was: {word}. Click New Game.", "bad")

    return (
        word, list(guessed), list(wrong), lives, over, hint_used,
        word_html,
        svg_frame(stage),
        lives_badge(lives),
        msg,
        wrong_line(wrong),
        ""
    )

def use_hint(word, guessed, wrong, lives, over, hint_used):
    guessed, wrong = safe_sets(guessed, wrong)

    if not word or not isinstance(word, str):
        return (
            word, list(guessed), list(wrong), lives, over, hint_used,
            "<div class='word-display'>_ _ _ _</div>",
            svg_frame(0),
            lives_badge(4),
            status_banner("Click New Game first.", "info"),
            wrong_line(wrong),
            ""
        )

    if over:
        stage = stage_from_lives(lives)
        return (
            word, list(guessed), list(wrong), lives, True, hint_used,
            f"<div class='word-display'>{build_display(word, guessed)}</div>",
            svg_frame(stage),
            lives_badge(lives),
            status_banner("Game over. Start a new game to use hint.", "info"),
            wrong_line(wrong),
            ""
        )

    if hint_used:
        stage = stage_from_lives(lives)
        return (
            word, list(guessed), list(wrong), lives, False, True,
            f"<div class='word-display'>{build_display(word, guessed)}</div>",
            svg_frame(stage),
            lives_badge(lives),
            status_banner("Hint already used.", "info"),
            wrong_line(wrong),
            ""
        )

    remaining = [ch for ch in set(word) if ch.isalpha() and ch not in guessed]
    if remaining:
        guessed.add(random.choice(remaining))

    hint_used = True
    stage = stage_from_lives(lives)
    word_html = f"<div class='word-display'>{build_display(word, guessed)}</div>"

    if all(ch in guessed for ch in word):
        over = True
        msg = status_banner("You won. Click New Game.", "good")
    else:
        msg = status_banner("Hint used.", "info")

    return (
        word, list(guessed), list(wrong), lives, over, hint_used,
        word_html,
        svg_frame(stage),
        lives_badge(lives),
        msg,
        wrong_line(wrong),
        ""
    )

# ---------- UI ----------
css = """
:root{
  --bg:#070a16;
  --panel:rgba(15, 23, 42, .62);
  --border:rgba(255,255,255,.10);
  --text:#e5e7eb;
  --muted:#94a3b8;
}

#app{ max-width: 1040px; margin: 0 auto; }
.gradio-container{
  background: radial-gradient(1200px 650px at 20% 0%, #1f2a5a 0%, var(--bg) 50%, #050814 100%) !important;
  color: var(--text) !important;
}

.card{
  border: 1px solid var(--border);
  border-radius: 18px;
  padding: 14px;
  background: var(--panel);
  box-shadow: 0 10px 26px rgba(0,0,0,.32);
}

.hang-card{
  border: 1px solid var(--border);
  border-radius: 18px;
  padding: 10px;
  background: rgba(255,255,255,.92);
  box-shadow: 0 10px 26px rgba(0,0,0,.32);
}

.header{
  display:flex;
  align-items:center;
  justify-content:space-between;
  gap:12px;
}
.header h1{
  margin: 0;
  font-size: 28px;
  font-weight: 900;
  color: var(--text);
}

.lives-badge{
  display:flex;
  align-items:center;
  gap:10px;
  padding: 8px 12px;
  border: 1px solid var(--border);
  border-radius: 999px;
  background: rgba(2,6,23,.35);
}
.lives-label{ color: var(--muted); font-size: 12px; font-weight: 700; text-transform: uppercase; letter-spacing: .08em; }
.lives-hearts{ font-size: 18px; }

.word-display{
  color: var(--text);
  font-size: 38px;
  font-weight: 900;
  letter-spacing: 10px;
  text-align: center;
  padding: 10px 0 6px;
}

.status{
  margin-top: 8px;
  border: 1px solid var(--border);
  border-radius: 14px;
  padding: 10px 12px;
  font-weight: 750;
  color: var(--text);
}
.status-info{ background: rgba(96,165,250,.12); }
.status-good{ background: rgba(34,197,94,.14); }
.status-bad{ background: rgba(239,68,68,.14); }

.wrong-title{ color: var(--muted); font-size: 12px; font-weight: 700; text-transform: uppercase; letter-spacing: .08em; }
.wrong-value{ color: var(--text); margin-top: 6px; font-size: 16px; font-weight: 700; }
.wrong-space{ height: 14px; }

button.primary{
  background: linear-gradient(135deg, #22c55e 0%, #16a34a 100%) !important;
  border: none !important;
}
button.secondary{
  background: linear-gradient(135deg, #60a5fa 0%, #2563eb 100%) !important;
  border: none !important;
}
"""

theme = gr.themes.Soft(primary_hue="green", secondary_hue="blue", neutral_hue="slate")

with gr.Blocks(css=css, theme=theme) as demo:
    with gr.Row(elem_classes=["card"]):
        gr.HTML("<div class='header'><h1>Hangman Game</h1></div>")
        lives_html = gr.HTML(lives_badge(4))

    with gr.Row():
        # LEFT: gameplay + hint BELOW wrong guesses
        with gr.Column(scale=3, elem_classes=["card"]):
            word_html = gr.HTML("<div class='word-display'>_ _ _ _</div>")
            status_html = gr.HTML(status_banner("Click New Game to start.", "info"))
            guess_inp = gr.Textbox(label="Your Guess", placeholder="Type one letter and press Enter")
            guess_btn = gr.Button("Guess", elem_classes=["primary"])
            wrong_html = gr.HTML(wrong_line(set()))
            hint_btn = gr.Button("Hint", elem_classes=["secondary"])  # moved here

        # RIGHT: hangman + difficulty + new game
        with gr.Column(scale=2):
            hang_html = gr.HTML(value=svg_frame(0))
            with gr.Column(elem_classes=["card"]):
                difficulty = gr.Dropdown(
                    ["Easy (4–6)", "Medium (7–9)", "Hard (10+)"],
                    value="Easy (4–6)",
                    label="Difficulty"
                )
                new_btn = gr.Button("New Game", elem_classes=["secondary"])

    # state
    s_word = gr.State(None)
    s_guessed = gr.State([])
    s_wrong = gr.State([])
    s_lives = gr.State(4)
    s_over = gr.State(False)
    s_hint_used = gr.State(False)

    demo.load(
        init_ui,
        inputs=[],
        outputs=[
            s_word, s_guessed, s_wrong, s_lives, s_over, s_hint_used,
            word_html, hang_html, lives_html, status_html, wrong_html, guess_inp
        ],
    )

    new_btn.click(
        new_game,
        inputs=[difficulty],
        outputs=[
            s_word, s_guessed, s_wrong, s_lives, s_over, s_hint_used,
            word_html, hang_html, lives_html, status_html, wrong_html, guess_inp
        ],
    )

    hint_btn.click(
        use_hint,
        inputs=[s_word, s_guessed, s_wrong, s_lives, s_over, s_hint_used],
        outputs=[
            s_word, s_guessed, s_wrong, s_lives, s_over, s_hint_used,
            word_html, hang_html, lives_html, status_html, wrong_html, guess_inp
        ],
    )

    guess_btn.click(
        guess_letter,
        inputs=[guess_inp, s_word, s_guessed, s_wrong, s_lives, s_over, s_hint_used],
        outputs=[
            s_word, s_guessed, s_wrong, s_lives, s_over, s_hint_used,
            word_html, hang_html, lives_html, status_html, wrong_html, guess_inp
        ],
    )

    guess_inp.submit(
        guess_letter,
        inputs=[guess_inp, s_word, s_guessed, s_wrong, s_lives, s_over, s_hint_used],
        outputs=[
            s_word, s_guessed, s_wrong, s_lives, s_over, s_hint_used,
            word_html, hang_html, lives_html, status_html, wrong_html, guess_inp
        ],
    )

demo.launch()

/tmp/ipython-input-3584434344.py:354: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(css=css, theme=theme) as demo:
/tmp/ipython-input-3584434344.py:354: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(css=css, theme=theme) as demo:


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c27e9a56b402a18c69.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
